In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  23.70 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  36.89 MB/s
Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  49.77 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.76 MB/s
vincent-0.4.4- 100% |###################

In [3]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import lxml.html as lh
import numpy as np

In [17]:
dfs = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M",header=0)
df = dfs[0]
print("Size of dataframe before:",df.shape)
df.head()
df.drop(df[df.Borough == 'Not assigned'].index,inplace=True)
df.index=range(len(df.index))
df.head(10)

#assign borough to neighborhood where unassigned
df["Neighbourhood"] = np.where(df["Neighbourhood"] == 'Not assigned', df['Borough'], df["Neighbourhood"])
df.head(10)

#group by postcode
grouped = df.groupby(['Postcode','Borough']).agg(lambda col: ','.join(col))
grouped.head()

Size of dataframe before: (289, 3)


,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge,Malvern"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [18]:
# number of rows of the dataframe
print("Size of dataframe after:", df.shape)

Size of dataframe after: (212, 3)


In [20]:
# get coordinates
!wget -q -O 'geo.csv' http://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [21]:
geo_df = pd.read_csv('geo.csv')
geo_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [30]:
geo_df.dtypes

Postcode      object
Latitude     float64
Longitude    float64
dtype: object

In [24]:
# clean up csv file
geo_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)

In [33]:
# combine the two dataframes
stack_df = pd.merge(df,
                 geo_df,
                 on='Postcode')
stack_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
